载入并消化文档后，下一步是将信息向量化，以便下一步检索。通常来说，调用向量化模型的工具整合在 `langchain_community` 包里，当然也有一些合作方单独开发了库整合了自己的全部工具。本文介绍两个主流的 embedding 工具 `HuggingFaceEmbeddings` 和 `OpenAIEmbeddings`，其它工具可参考 https://python.langchain.com/api_reference/community/index.html。

In [4]:
import os
from langchain.document_loaders import TextLoader

root = os.getcwd()
loader = TextLoader(os.path.join(root, 'data/paul_graham/paul_graham_essay.txt'))
data = loader.load()

# `HuggingFaceBgeEmbeddings`
在使用这个类之前需要安装 `langchain-huggingface` 包。这个类其实是 `sentence_transformers.SentenceTransformer` 的封装。主要参数有：
- `cache_folder`：模型的缓存路径，也可以设置 `SENTENCE_TRANSFORMERS_HOME` 环境变量
- `model_name`：模型名称，比如 `BAAI/bge-small-zh-v1.5`
- `multi_process`：是否使用多个 GPU
- `show_progress`：是否显示进度条
- `encode_kwargs`：传递给这个类的 encoding 参数，继承自 `sentence_transformers.SentenceTransformer`。
- `model_kwargs`：传递给这个类的参数，继承自 `sentence_transformers.SentenceTransformer`，比较重要的参数有 `device` 和 `trust_remote_code`
- `query_encode_kwargs`：同样是传递给这个类的参数，继承自 `sentence_transformers.SentenceTransformer`

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')

/Users/wenjiazhai/miniconda3/envs/langchain/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


初始化后可用的方法有：
- `embed_documents`：接受一个字符串列表并向量化
- `aembed_documents`：上一个方法的异步版本
- `embed_query`：接受一个字符串并向量化
- `aembed_query`：上一个方法的异步版本

In [6]:
embed_content = embedding.embed_documents([d.page_content for d in data])
print('dimension for embedded docs:', len(embed_content[0]))

embed_query = embedding.embed_query("What did the author do growing up?")
print('dimension for embedded query:', len(embed_query))

dimension for embedded docs: 384
dimension for embedded query: 384


`embed_documents` 和 `embed_query` 的区别在于，前者接受一个字符串列表，后者接受一个字符串。如果无法连接 HuggingFace，可以使用国内的 `langchain_community.embeddings.modelscope_hub.ModelScopeEmbeddings`，参数有 `model_id` 和 `model_revision`，不再赘述。

# `OpenAIEmbeddings`
这个类保存在 `langchain-openai` 包中，使用需要安装 `langchain-openai` 和 `openai`。这个向量化工具需要 OpenAI 的 key 来运行。提供 `OPENAI_API_KEY` 有若干方法：
1. 通过环境变量：
```python
import os
os.environ["OPENAI_API_KEY"] = "sk-..."
```
2. 通过参数：
```python
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key="sk-...")
```
3. 通过 `.env` 文件：
```python
# pip install python-dotenv
from dotenv import load_dotenv  # 用于加载环境变量
load_dotenv()  # 加载 .env 文件中的环境变量
```
4. 运行时输入：
```python
import openai

openai.api_key = input("请输入您的 OpenAI API 密钥: ") # 提示用户输入 OpenAI API 密钥
```
目前 OpenAI 支持的模型有 `text-embedding-ada-002`， `text-embedding-3-small` 和 `text-embedding-3-large`。`OpenAIEmbeddings` 的主要参数有
- `model`：模型名称
- `api_key`：key 值

In [2]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small',
                             openai_api_key="sk-...")

embed = embedding.embed_query("Hello world")
print(len(embed))

1536
